In [1]:
import pint
import os, sys
from utils import *

lib_path = get_project_root()
sys.path.insert(0, lib_path)

from lib.calculator import LinkBudgetCalculator

ModuleNotFoundError: No module named 'ephem'

In [ ]:
ureg = pint.UnitRegistry()

# create the linkBudgetCalculator object
lb_calc = LinkBudgetCalculator(ureg)

## Analysis of Design Iteration 1

### Description

First design iteration of Mach 30 Groundsphere used to capture and demodulate NOAA satellite imagery broadcasted in an APT (Automatic Picture Transmission) format. This is the simplest possible iteration, cosisting of an eggbeater antenna and a software defined radio. The eggbeater antenna will provide preferable directivyty torwards zenith, while rejecting signals orginating near the horizon. The flexibility of software defined radios allows us to ideally include the RF front end within the demodulator, but practical considerations related to performance may require external amplifiers or filters in future design iterations. Performance of this system is explored using the python link budget library designed for this project and pyephem for satellite trajectory passover calculations.  

![alt text](block_diagram_template.png "Block Diagram")
*<p style="text-align: center;">Block Diagram</p>*


### Setup of Ground Station Values

Ground station will have a custom-designed eggbeater antenna, followed by a short, low-loss RF coaxial cable, and an RTL-SDR (Software Defined Radio). The eggbeater antenna is to be designed to have a center frequency of 137.5 MHz. A 

Values for specific ground station are below:

In [ ]:
lb_calc.altitude_ground_station   =  400 * ureg.meter
lb_calc.implementation_loss       = -1.0   # dB
lb_calc.polarization_losses       =  0.0   # dB
lb_calc.receive_antenna_gain      =  5.4   # dB
lb_calc.system_noise_figure       =  5.0   # dB

### Setup of Satellite Values

Values stated for a certain NOAA satellite. These values are covered in the documentation.

In [ ]:
lb_calc.altitude_satellite        =  860 * ureg.kilometer
lb_calc.transmit_power            =  5.0 * ureg.watt
lb_calc.transmit_losses           = -1.0   # dB
lb_calc.transmit_antenna_gain     =  4.0   # dBi

### Setup of Additional Values

Create functions for the changing elevation angle and atmospheric loss values. Emperical loss table for antenna pointing loss with respect to elevation angle provided below.

In [ ]:
# receive pointing loss for given elevation (at our frequency)
# these values are different for each antenna system, so the values are defined here for clarity
# see referenced antenna specifications for more information
def rx_pointing_loss_at_elev(elev):
    return -10.0 if elev < 5  * ureg.degrees else \
           -6.0  if elev < 20 * ureg.degrees else \
           -3.0  if elev < 35 * ureg.degrees else \
           -1.0  if elev < 50 * ureg.degrees else 0.0

Set paramaters for the analysis here

Additional values are needed for the channel characteristics. A few of these values will change during an orbital pass!

In [ ]:
lb_calc.orbit_elevation_angle     =  0.001          * ureg.degrees
lb_calc.downlink_frequency        =  137.5          * ureg.megahertz
lb_calc.target_energy_noise_ratio =  20.0             # dB
lb_calc.noise_bandwidth           =  34.0           * ureg.kilohertz
lb_calc.transmit_pointing_loss    = -3.0              # dB
lb_calc.atmospheric_loss          =  atmloss_at_elev(ureg, lb_calc.orbit_elevation_angle)
lb_calc.receiving_pointing_loss   =  rx_pointing_loss_at_elev(lb_calc.orbit_elevation_angle)

### Analysis

Data for NOAA satellite TLEs obtained from public libraries such as Pyephem used in link budget calculations.

Increment through an entire orbital pass.

In [ ]:
# get the elevation angles of different pass types using a TLE
# use an old TLE (line0, line1, line2)
tle_old = ('NOAA 19 [+]', '1 33591U 09005A   18092.90091581  .00000055  00000-0  55075-4 0  9994',
       '2 33591  99.1353  69.4619 0014005 174.2137 185.9198 14.12266303471284')
results = []
# poor pass
results.append(compute_angles(tle=tle_old, date='2018/04/03 02:00.00'))
# average pass
results.append(compute_angles(tle=tle_old, date='2018/04/03 10:00.00'))
# excellent pass
results.append(compute_angles(tle=tle_old, date='2018/04/01 06:00.00'))

outputs = []
outputs.append([])
outputs.append([])
outputs.append([])

for n, res in enumerate(results):
    for i, angle in enumerate(res[1]):
        # compute the budget
        lb_calc.run()

        # ensure the computation was valid
        if not lb_calc.is_valid:
            raise Exception('Run at elevation angle ', lb_calc.orbit_elevation_angle, ' was not valid')

        # save output as a tuple to output list
        outputs[n].append( (lb_calc.link_distance, \
                         lb_calc.downlink_path_loss, \
                         lb_calc.received_power, \
                         lb_calc.energy_noise_ratio, \
                         lb_calc.link_margin) )

        # increment the elevation angle
        lb_calc.orbit_elevation_angle     = angle * ureg.degrees
        lb_calc.atmospheric_loss          = atmloss_at_elev(ureg, lb_calc.orbit_elevation_angle)
        lb_calc.receiving_pointing_loss   = rx_pointing_loss_at_elev(lb_calc.orbit_elevation_angle)

# DEBUG
print('Max poor elevation angle: {}'.format(max(results[0][1])))
print('Duration of poor pass: {}'.format(results[0][2][-1] - results[0][2][0]))
print('Max average elevation angle: {}'.format(max(results[1][1])))
print('Duration of average pass: {}'.format(results[1][2][-1] - results[1][2][0]))
print('Max good elevation angle: {}'.format(max(results[2][1])))
print('Duration of good pass: {}'.format(results[2][2][-1] - results[2][2][0]))

Once analysis has run, examine the results.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# extract lists of the variables
distances = []
loss = []
rx_pow = []
eb_no = []
margins = []
for j in range(0, 3):
    distances.append([ i[0].magnitude for i in outputs[j] ])
    loss.append(     [ i[1] for i in outputs[j] ])
    rx_pow.append(   [ i[2] for i in outputs[j] ])
    eb_no.append(    [ i[3] for i in outputs[j] ])
    margins.append(  [ i[4] for i in outputs[j] ])

In [ ]:
fig = []
types = ['Poor', 'Average', 'Good']

for i in range(0, len(outputs)):
    fig.append(plt.figure())
    p = fig[i].add_subplot(111)
    p.plot(range(0, len(margins[i])), margins[i], 'b', range(0, len(margins[i])), [0] * len(margins[i]), 'r--')
    plt.title('Link Margins for Example {} Pass'.format(types[i]))
    plt.xlabel('Time (s)')
    plt.ylabel('Margin (dB)')

    total_s = 0
    for m in margins[i]:
        if m > 0:
            total_s += 1

    print('Duration for {} pass above margin: {} seconds ({:.3} minutes)'.format(types[i], total_s, total_s/60))

### Conclusions

Analysis of this design iteration has yielded the following results. Given the antenna gain values, system losses, and receiver amplifier gain, this system is capable of demodulating NOAA APT signals for good and average passes. The plots generated above display how link margin over the duration of a satellite passover. Increasing the gain or reducing the noise in the system may improve the performance of the system.
